In [15]:
#출처 : https://oboki.net/workspace/programming/language/python/folium-%EC%A7%80%EB%A6%AC%EC%A0%95%EB%B3%B4-%EC%8B%9C%EA%B0%81%ED%99%94/

### GIS 시각화

In [11]:
import pandas as pd

# data 불러오기

df = pd.read_csv('./report/190718_COUPON_CUST.csv', encoding = 'cp949', low_memory=False)
GIS = df[['LAT', 'LON']].dropna()

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


FileNotFoundError: [Errno 2] File b'./report/190718_COUPON_CUST.csv' does not exist: b'./report/190718_COUPON_CUST.csv'

In [ ]:
import folium
from folium.plugins import MarkerCluster
 
# map 처음 표기위치 설정
map1 = folium.Map(location=[37.543083,126.977378],zoom_start=11) # 서울
# map1 = folium.Map(location=[36.5053542,127.7043419],zoom_start=8) # 한국

# 지도에 각각의 지역 표기
marker_cluster = MarkerCluster().add_to(map1)

for i in range(len(GIS)):
    folium.Marker(
    location=[GIS['LAT'].iloc[i] ,GIS['LON'].iloc[i]],
    icon=folium.Icon(color='red',icon='ok'),
  ).add_to(marker_cluster)

In [ ]:
# 지도 표기
map1

In [ ]:
# 지도 저장 : html
# map1.save('map.html')

In [2]:
import folium
from folium.plugins import MarkerCluster

In [3]:
map1 = folium.Map(location = [37.558079, 126.907864] , zoom_start = 15)
map1

In [ ]:
y = datas_geo.geo_y.values
x = datas_geo.geo_x.value
location = list(zip(y,x))
popups = list(datas_geo.title.valuse)

icon_create_function = """\
function(cluster) {
    retuen L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className : 'maker-cluster maker-cluster-large',
    iconSize : new L.Point(30,30)
    });
}"""




In [9]:
import folium
 
map1 = folium.Map(
  location=[36.5053542,127.7043419],
  zoom_start=8
)
 
lotto = [
  {"store":"담배","loc":[37.62585944357624,127.01847823823798]},   {"store":"화곡본마트","loc":[37.54248103738589,126.84414659211494]},   {"store":"용꿈돼지꿈","loc":[37.5447438346992,126.95223862043447]},   {"store":"일이오마켓","loc":[37.47943673430032,126.98346178441342]},   {"store":"여명슈퍼마켓","loc":[37.61432815661055,127.0415039221072]},   {"store":"5가로또레드탑","loc":[37.570756251987575,127.00235901546226]},   {"store":"복권세상","loc":[35.91885025656432,128.55034430109828]},   {"store":"복권전문점","loc":[37.50384327747725,126.71513104367074]},   {"store":"인현동지하가판","loc":[37.47627910055104,126.63147671205866]},   {"store":"CU(광주서동점)","loc":[35.14814807640179,126.90555219336346]},   {"store":"지산로또방","loc":[35.148338888178,126.93203920504686]},   {"store":"천사로또방","loc":[37.63618838692431,127.21107884503209]},   {"store":"오렌지통신","loc":[37.42889361993242,127.10232604117991]},   {"store":"여수복권방","loc":[37.418353650942116,127.12640523182601]},   {"store":"둘리복권방","loc":[37.34532813934063,126.73627994929885]},   {"store":"금성24시편의점","loc":[37.84401249737699,127.06236893072152]},   {"store":"GS25(청주주은점)","loc":[36.60987111587086,127.49134026856717]},   {"store":"장미슈퍼","loc":[36.276304155315685,126.90940713047799]},   {"store":"장미슈퍼","loc":[36.276304155315685,126.90940713047799]},   {"store":"아이24(수송점)","loc":[35.970108052787594,126.7189421934312]},   {"store":"탑로또","loc":[34.87844231008963,128.62803523643754]} 
]
 
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(map1)
 
for i in range(len(lotto)):
  folium.Marker(
    location=lotto[i]['loc'],
    popup=lotto[i]['store'],
    icon=folium.Icon(color='red',icon='ok'),
  ).add_to(marker_cluster)


In [2]:
 
 
lotto

[{'store': '담배', 'loc': [37.62585944357624, 127.01847823823798]},
 {'store': '화곡본마트', 'loc': [37.54248103738589, 126.84414659211494]},
 {'store': '용꿈돼지꿈', 'loc': [37.5447438346992, 126.95223862043447]},
 {'store': '일이오마켓', 'loc': [37.47943673430032, 126.98346178441342]},
 {'store': '여명슈퍼마켓', 'loc': [37.61432815661055, 127.0415039221072]},
 {'store': '5가로또레드탑', 'loc': [37.570756251987575, 127.00235901546226]},
 {'store': '복권세상', 'loc': [35.91885025656432, 128.55034430109828]},
 {'store': '복권전문점', 'loc': [37.50384327747725, 126.71513104367074]},
 {'store': '인현동지하가판', 'loc': [37.47627910055104, 126.63147671205866]},
 {'store': 'CU(광주서동점)', 'loc': [35.14814807640179, 126.90555219336346]},
 {'store': '지산로또방', 'loc': [35.148338888178, 126.93203920504686]},
 {'store': '천사로또방', 'loc': [37.63618838692431, 127.21107884503209]},
 {'store': '오렌지통신', 'loc': [37.42889361993242, 127.10232604117991]},
 {'store': '여수복권방', 'loc': [37.418353650942116, 127.12640523182601]},
 {'store': '둘리복권방', 'loc': [37.

In [10]:
# 지도 표기
map1

